# RAG from scratch

- Adapted from https://github.com/opendatahub-io/llama-stack-demos/
- Requires ollama template 

In [1]:
import rich
import os
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient, RAGDocument


# Variables

In [ ]:
load_dotenv()
base_url=os.getenv("BASE_URL") #local "http://localhost:8321"
chunk_size_in_tokens=os.getenv("CHUNK_SIZE_TOKEN") #512
collection_name=os.getenv('COLLECTION_NAME')   #vector db collection name p.e : raptormcl
embedding_model_name=os.getenv('EMBEDDING_MODEL_NAME')  #p.e. "mxbai-embed" 
embedding_size=int(os.getenv('EMBEDDING_SIZE'))   #vector size embedding corresponding to embedding model used p.e. 1024
chunk_size=int(os.getenv('CHUNK_SIZE'))  #chunk size p.e. 4000


In [ ]:
# base_url="http://localhost:8321"
# chunk_size_in_tokens=512


In [3]:
client = LlamaStackClient(
    base_url=base_url,
)

In [4]:
client.models.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[Model(identifier='llama3.2:3b-instruct-fp16', metadata={}, api_model_type='llm', provider_id='ollama', type='model', provider_resource_id='llama3.2:3b-instruct-fp16', model_type='llm'),
 Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', type='model', provider_resource_id='all-minilm:latest', model_type='embedding')]

In [5]:
from llama_stack_client import LlamaStackClient, RAGDocument

pdf_urls = [
    "https://arxiv.org/pdf/2304.08641.pdf",
    "https://www.cs.cmu.edu/~tom/mlbook/NBayesLogReg.pdf",
]

documents = [
    RAGDocument(
        document_id=f"pdf-{i}",
        content=url,  # still use `content` for the URL
        mime_type="application/pdf",
        metadata={"source": "arxiv" if "arxiv" in url else "cmu"},
    )
    for i, url in enumerate(pdf_urls)
]


In [ ]:
vector_db_id = collection_name
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=embedding_model_name,
    embedding_dimension=384,
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/vector-dbs "HTTP/1.1 200 OK"


VectorDBRegisterResponse(embedding_dimension=384, embedding_model='all-MiniLM-L6-v2', identifier='demoInnovate-vectordb', provider_id='faiss', type='vector_db', provider_resource_id='demoInnovate-vectordb', owner={'principal': '', 'attributes': {}})

Ingesting documents into a vector database:

In [9]:
rich.print(documents)

[
    {
        'document_id': 'num-0',
        'content': 'https://www.openshift.guide/openshift-guide-screen.pdf',
        'mime_type': 'application/pdf',
        'metadata': {}
    }
]

In [10]:
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/insert "HTTP/1.1 200 OK"


In [11]:
prompt = "What are the top 5 topics that were explained? Only list succinct bullet points."

In [12]:
# higher level tool provides packaged results, can span multiple dbs
tool_response = client.tool_runtime.rag_tool.query(
    content=prompt, vector_db_ids=[vector_db_id]
)
rich.print(tool_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/query "HTTP/1.1 200 OK"


QueryResult(
    metadata={'document_ids': ['num-0', 'num-0', 'num-0', 'num-0', 'num-0']},
    content=[
        TextContentItem(
            text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
            type='text'
        ),
        TextContentItem(
            text="Result 1\nContent: \ntemplates, 32\nTopology, 27\nTwelve-Factor App, 21, 40\nTypeScript, 
24\nU\nUBI, 19\nV\nVertical scaling, 42\nVisual Studio Code, 14\nW\nWeb Console, 22, 40\n49\nMetadata: 
{'document_id': 'num-0'}\n",
            type='text'
        ),
        TextContentItem(
            text="Result 2\nContent: author are affiliated with Red Hat, Inc. or any of its subsidiaries in any 
way.\nOpenShift®, Red Hat®, and the Red Hat logo are trademarks or registered trademarks of Red Hat,\nInc. or its 
subsidiaries in the United States and other countries. All other trademarks are the\nproperty of their respective 
owners.\nToolchain\nThis website and associated content in PDF, EPUB, and man page formats have been created 
using\nthe following tools:\n• Antora\n• Asciidoctor\n• Search engine powered by TNTSearch and SQLite\n• Website 
built and deployed from a GitLab CI/CD pipeline.\n47\nIndex\n@\n.NET, 14, 24\nA\nAdministration perspective, 
40\nArgo CD, 28\nASP.NET, 44\nAWS, 8\nAWS Lambda, 34\nAzure, 8\nAzure Functions, 34\nB\nBuildConfig, 10\nC\nC#, 
44\nC++, 14\nCI/CD, 28\nCode-Ready Containers, 8\nCodeReady Containers, 15\ncommand-line tools, 11\ncontainer, 
22\ncontainer image, 19\ncontainer registry, 24\nCRC, 8, 15\nCustom Resource Definitions, 9\nD\nDeploymentConfig, 
9\nDeveloper Catalog, 22\nDeveloper perspective, 22, 25\nDeveloper Sandbox for Red Hat OpenShift, 
13\nE\nElasticSearch, 40\nEnvoy, 37\nF\nFlux, 28\nG\nGit, 25\nGit repository, 22\nGitHub, 4\nGo, 14, 24\nGoogle, 
34\nGoogle Cloud Platform, 8\nGrafana, 45\nH\nHorizontal scaling, 42\nI\nIBM Cloud, 8\nImporting YAML, 23\ninstall 
OpenShift Local, 16\nJ\nJAR file, 23\nJava, 14, 24, 44\nJavaScript, 24, 44\nJenkins, 28\nK\nKiali, 36\nKibana, 40, 
40\nKnative, 34\nKubernetes, 7\nL\nlogs, 40\nM\nMicroservices, 36\nmonitor, 40\nN\nNode.js, 14\nnon-root accounts, 
20\nO\nOpenShift 4.12, 33\nOpenShift Kubernetes Distribution, 8\nMetadata: {'document_id': 'num-0'}\n",
            type='text'
        ),
        TextContentItem(
            text="Result 3\nContent:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . \xa02\n1.4. Example Code . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa03\n1.5. Recommended LinkedIn Learning 
Courses . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa03\n2. 
GitHub Repository . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . \xa04\n2.1. Branches . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
\xa04\n2.2. Beware! . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa04\nGetting Started. . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\nMetadata: 
{'document_id': 'num-0'}\n",
            type='text'
        ),
        TextContentItem(
            text="Result 4\nContent: 23\ninstall OpenShift Local, 16\nJ\nJAR file, 23\nJava, 14, 24, 
44\nJavaScript, 24, 44\nJenkins, 28\nK\nKiali, 36\nKibana, 40, 40\nKnative, 34\nKubernetes, 7\nL\nlogs, 
40\nM\nMicroservices, 36\nmonitor, 40\nN\nNode.js, 14\nnon-root accounts, 20\nO\nOpenShift 4.12, 33\nOpenShift 
Kubernetes Distribution, 8\nOpenShift Service Mesh, 36\noperator, 28, 36\nOperatorHub, 33, 36\nOperators, 
33\n48\nP\nperspecti

In [13]:
# we can also query the vector db directly
db_response = client.vector_io.query(
    vector_db_id=vector_db_id,
    query=prompt,
)
rich.print(db_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/vector-io/query "HTTP/1.1 200 OK"


QueryChunksResponse(
    chunks=[
        Chunk(
            content='\ntemplates, 32\nTopology, 27\nTwelve-Factor App, 21, 40\nTypeScript, 24\nU\nUBI, 
19\nV\nVertical scaling, 42\nVisual Studio Code, 14\nW\nWeb Console, 22, 40\n49',
            metadata={'document_id': 'num-0', 'token_count': 64.0, 'metadata_token_count': 1.0},
            embedding=[
                -0.020887844,
                -0.009134637,
                -0.05088005,
                -0.03797599,
                0.01051593,
                0.054153327,
                -0.07487062,
                0.11042954,
                -0.08174462,
                0.032844923,
                -0.030411718,
                0.014279036,
                -0.031293657,
                -0.013851491,
                0.06904846,
                0.025445996,
                -0.05622258,
                -0.018301316,
                0.059643798,
                -0.02672893,
                0.01959039,
                -0.031798042,
                -0.041182466,
                -0.03657983,
                -0.006874671,
                0.057913844,
                -0.057438046,
                -0.015592471,
                0.09899554,
                -0.13055278,
                -0.037659716,
                0.0058229477,
                0.0735941,
                0.025065878,
                0.023600334,
                -0.036850076,
                0.026274405,
                -0.079721235,
                -0.087336466,
                -0.025518935,
                -0.08680117,
                0.030678704,
                -0.023642104,
                0.061358284,
                0.034251824,
                -0.10111537,
                -0.1013808,
                -0.030249609,
                -0.053116594,
                0.003038648,
                -0.04133704,
                -0.037443385,
                -0.04654735,
                -0.0021673208,
                0.05269337,
                -0.030280814,
                -0.0006071019,
                0.026407924,
                0.03682606,
                -0.03225201,
                0.02500918,
                -0.003963993,
                -0.042028505,
                0.030061217,
                0.011108608,
                0.03991852,
                -0.000689326,
                0.045123555,
                0.04956153,
                -0.04393118,
                -0.07698064,
                -0.060855076,
                -0.024793327,
                0.0147253275,
                -0.033356685,
                -0.074074246,
                -0.01901284,
                -0.054765776,
                -0.007212972,
                -0.06411081,
                -0.022898717,
                0.0007510862,
                0.0017773749,
                0.11067964,
                0.024626967,
                0.040484514,
                0.038124975,
                -0.013740793,
                0.041543014,
                0.033232205,
                -0.015544788,
                -0.069006465,
                0.062012874,
                0.04880897,
                -0.035550196,
                -0.015089697,
                0.024411425,
                -0.062503055,
                -0.012469489,
                0.06435011,
                -0.0154851945,
                -0.05145209,
                0.09578783,
                0.015951637,
                -0.051247895,
                0.039537806,
                0.077087425,
                0.022182243,
                -0.020406526,
                -0.03700791,
                -0.0451227,
                0.020739935,
                -0.08778193,
                -0.1145982,
                0.068037614,
                -0.06917088,
                -0.006452935,
                -0.03763229,
                0.097271025,
                0.069599,
                0.020399483,
                0.032017235,
                -0.035780113,
                -0.063457705,
                -

In [14]:
# prompt_context = tool_response.content
prompt_context = "\n".join([c.content for c in db_response.chunks])

In [15]:
messages = [{"role": "system", "content": "You are a helpful assistant."}]
extended_prompt = f"""
Please answer the given query using the context below.

QUERY:
{prompt}

CONTEXT:
{prompt_context}
"""
messages.append({"role": "user", "content": extended_prompt})
rich.print(messages)

[
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {
        'role': 'user',
        'content': '\nPlease answer the given query using the context below.\n\nQUERY:\nWhat are the top 5 topics 
that were explained? Only list succinct bullet points.\n\nCONTEXT:\n\ntemplates, 32\nTopology, 27\nTwelve-Factor 
App, 21, 40\nTypeScript, 24\nU\nUBI, 19\nV\nVertical scaling, 42\nVisual Studio Code, 14\nW\nWeb Console, 22, 
40\n49\nauthor are affiliated with Red Hat, Inc. or any of its subsidiaries in any way.\nOpenShift®, Red Hat®, and 
the Red Hat logo are trademarks or registered trademarks of Red Hat,\nInc. or its subsidiaries in the United States
and other countries. All other trademarks are the\nproperty of their respective owners.\nToolchain\nThis website 
and associated content in PDF, EPUB, and man page formats have been created using\nthe following tools:\n• 
Antora\n• Asciidoctor\n• Search engine powered by TNTSearch and SQLite\n• Website built and deployed from a GitLab 
CI/CD pipeline.\n47\nIndex\n@\n.NET, 14, 24\nA\nAdministration perspective, 40\nArgo CD, 28\nASP.NET, 44\nAWS, 
8\nAWS Lambda, 34\nAzure, 8\nAzure Functions, 34\nB\nBuildConfig, 10\nC\nC#, 44\nC++, 14\nCI/CD, 28\nCode-Ready 
Containers, 8\nCodeReady Containers, 15\ncommand-line tools, 11\ncontainer, 22\ncontainer image, 19\ncontainer 
registry, 24\nCRC, 8, 15\nCustom Resource Definitions, 9\nD\nDeploymentConfig, 9\nDeveloper Catalog, 22\nDeveloper 
perspective, 22, 25\nDeveloper Sandbox for Red Hat OpenShift, 13\nE\nElasticSearch, 40\nEnvoy, 37\nF\nFlux, 
28\nG\nGit, 25\nGit repository, 22\nGitHub, 4\nGo, 14, 24\nGoogle, 34\nGoogle Cloud Platform, 8\nGrafana, 
45\nH\nHorizontal scaling, 42\nI\nIBM Cloud, 8\nImporting YAML, 23\ninstall OpenShift Local, 16\nJ\nJAR file, 
23\nJava, 14, 24, 44\nJavaScript, 24, 44\nJenkins, 28\nK\nKiali, 36\nKibana, 40, 40\nKnative, 34\nKubernetes, 
7\nL\nlogs, 40\nM\nMicroservices, 36\nmonitor, 40\nN\nNode.js, 14\nnon-root accounts, 20\nO\nOpenShift 4.12, 
33\nOpenShift Kubernetes Distribution, 8\n . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . \xa02\n1.4. Example Code . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa03\n1.5. Recommended LinkedIn Learning
Courses . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa03\n2. 
GitHub Repository . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . \xa04\n2.1. Branches . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
\xa04\n2.2. Beware! . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa04\nGetting Started. . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n'
    }
]

In [16]:
response = client.inference.chat_completion(
    messages=messages,
    model_id="llama3.2:3b-instruct-fp16",
    timeout=600
)
rich.print(response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/inference/chat-completion "HTTP/1.1 200 OK"


ChatCompletionResponse(
    completion_message=CompletionMessage(
        content='Here are the top 5 topics that were explained:\n\n• Kubernetes\n• Twelve-Factor App\n• 
TypeScript\n• UBI (Universal Basic Income)\n• Vertical scaling',
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    logprobs=None,
    metrics=[
        Metric(metric='prompt_tokens', value=1142.0, unit=None),
        Metric(metric='completion_tokens', value=44.0, unit=None),
        Metric(metric='total_tokens', value=1186.0, unit=None)
    ]
)

In [17]:
# Left as an extra exercise for the reader
from llama_stack_client import Agent, AgentEventLogger

rag_agent = Agent(
    client, 
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant",
    tools = [
        {
          "name": "builtin::rag/knowledge_search",
          "args" : {
            "vector_db_ids": [vector_db_id],
          }
        }
    ],
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Arag%2Fknowledge_search "HTTP/1.1 200 OK"


In [18]:
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
user_prompts = [
    "What is Lora?"
]
session_id = rag_agent.create_session(f"rag session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = rag_agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/8c4eeb3f-18e9-45f3-a684-6196aee7b466/session "HTTP/1.1 200 OK"


User> What is Lora?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/8c4eeb3f-18e9-45f3-a684-6196aee7b466/session/3694eefb-f650-40c2-bccd-a140553ed66a/turn "HTTP/1.1 200 OK"


inference> [knowledge_search(query="Lora")]
tool_execution> Tool:knowledge_search Args:{'query': 'Lora'}
tool_execution> Tool:knowledge_search Response:[TextContentItem(text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n', type='text'), TextContentItem(text="Result 1\nContent:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa010\n5. OpenShift-only Command-Line Tools. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa011\n5.1. oc . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa011\n5.2. odo . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa011\n6. Red Hat OpenShift Developer Sandbox . . . . . . . . . . . . . . . . 